## Upload images

In [5]:
import json
import time
from datetime import datetime
from os import path, getcwd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.support.ui import WebDriverWait

In [ ]:
driver = webdriver.Chrome(path.join(getcwd(), 'chromedriver'))
driver.get('http://103.250.185.39/~ioitcollege/corifycar/wp-login.php?loggedout=true&wp_lang=en_US')

In [ ]:
def log(message):
    now = str(datetime.now())[:-7]
    print(now, message, sep=' ')

In [ ]:
username, password = 'Corify_Car', input('Enter password for Corify_Car: ')
driver.find_element_by_xpath('//*[@id="user_login"]').send_keys(username)
driver.find_element_by_xpath('//*[@id="user_pass"]').send_keys(password)
driver.find_element_by_xpath('//*[@id="wp-submit"]').click()

In [ ]:
driver.get('http://103.250.185.39/~ioitcollege/corifycar/wp-admin/edit.php?post_type=services&mode=list')

In [3]:
cars = json.load(open('cars.json', 'r'))

In [ ]:
links = {}
for i in range(1, 165):
    title = driver.find_element_by_xpath(f'/html/body/div[1]/div[2]/div[2]/div[1]/div[5]/form[1]/table/tbody/tr[{i}]/td[1]/strong/a') 
    links[title.text] = title.get_attribute('href')

In [ ]:
i = 0
for model in links:        
    log(f'{i}. GET ' + model)

    driver.get(links[model])
    div = driver.find_element_by_xpath('/html/body/div[1]/div[2]/div[2]/div[1]/div[5]/form/div/div/div[2]/div/div[5]/div[2]')
    
    if div.text == 'Set featured image':
        driver.find_element_by_xpath('/html/body/div[1]/div[2]/div[2]/div[1]/div[5]/form/div/div/div[2]/div/div[5]/div[2]/p/a').click()
        time.sleep(0.5)
        driver.find_element_by_xpath('//*[@id="media-search-input"]').send_keys(model)
        time.sleep(3)
        try:
            img = driver.find_element_by_xpath('/html/body/div[14]/div[1]/div/div/div[3]/div[2]/div/ul/li/div')
            img.click()
            img.click()
        except:
            pass
        WebDriverWait(driver, 10).until(ec.element_to_be_clickable((By.XPATH, '/html/body/div[14]/div[1]/div/div/div[4]/div/div[2]/button'))).click()

        while True:
            try:
                driver.find_element_by_xpath('/html/body/div[1]/div[2]/div[2]/div[1]/div[5]/form/div/div/div[2]/div/div[5]/div[2]/p[2]')
                break
            except:
                pass

        div = driver.find_element_by_xpath('//*[@id="submitdiv"]')
        driver.execute_script("arguments[0].scrollIntoView();", div)
        driver.find_element_by_xpath('//*[@id="publish"]').click()

        j = 1
        while True:
            spinner = driver.find_element_by_xpath('//*[@id="publishing-action"]/span').get_attribute('class')
            log(spinner)
            if spinner == 'spinner' and j == 1:
                j = 2
            if spinner == 'spinner is-active':
                j = 3
            if spinner == 'spinner' and j == 3:
                log('Done')
                break
            time.sleep(0.5)

    i += 1